# Create a Perturbation in a Material

Create a perturbation in a material with a specified smooth function or a custom function described with [SymPy](https://docs.sympy.org/latest/tutorials/intro-tutorial/intro.html) expressions. 

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set perturbation parameters in cell 2.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create the Perturbation: Add a smooth perturbation to the material
2. Visualize the Perturbed Material

## Notes

1. For more information, see [Introduction](Introduction.ipynb)


## 1. Prepare the Environment
### 1.1. Set up supercell parameters 

In [1]:
SUPERCELL_MATRIX = [[30, 0, 0], [0, 30, 0], [0, 0, 1]] 

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [2]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("")

### 1.3. Get input materials
Materials are loaded with `get_materials()`.

In [3]:
from utils.jupyterlite import get_materials
materials = get_materials(globals())

0: Data from 0-Ni has been read successfully.
1: Data from 1-Graphene has been read successfully.
2: Data from 10-Al2O3, Sapphire, RHL (R-3c) 3D (Bulk), mp-1143 (1) has been read successfully.
3: Data from 11-ZnO, Zinc Oxide, HEX (P6_3mc) 3D (Bulk), mp-2133 has been read successfully.
4: Data from 12-Cd4 Te4 has been read successfully.
5: Data from 13-Si4 C4 has been read successfully.
6: Data from 14-GaN, Gallium Nitride, HEX (P6_3mc) 3D (Bulk), mp-804 has been read successfully.
7: Data from 15-WS2, Tungsten Disulfide, HEX (P-6m2) 2D (Monolayer), 2dm-3749 has been read successfully.
8: Data from 16 - In4 P4 has been read successfully.
9: Data from 17 - Silicon FCC (8) has been read successfully.
10: Data from 4-Te2Mo has been read successfully.
11: Data from 5-HfO2 has been read successfully.
12: Data from 6-Ni4(110), termination Ni_Pmmm_2, Slab, Terrace, 1 steps, [2 0 0] has been read successfully.
13: Data from 7-Ag4 has been read successfully.
14: Data from 8-Si, Silicene, HEX (P-

### 1.4. Create and preview Supercell

In [4]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

unit_cell = materials[0]
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

GridBox(children=(VBox(children=(Label(value='Ni900 - Material - rotation: 0x', layout=Layout(align_self='cent…

## 2. Create the SineWave Perturbation
### 2.1. Set sine wave perturbation parameters
Use perturbation function imported from `mat3ra.made.tools.utils.perturbation` folder.

In [5]:
from mat3ra.made.tools.build.perturbation import PerturbationConfiguration, SlabPerturbationBuilder
from mat3ra.made.tools.utils.perturbation import SineWavePerturbationFunctionHolder

amplitude = 0.05
wavelength = 1
phase = 0
axis = "y"
perturbation_function = SineWavePerturbationFunctionHolder(amplitude=amplitude, 
                                                           wavelength=wavelength, 
                                                           phase=phase,
                                                           axis=axis)

configuration = PerturbationConfiguration(material=supercell, 
                                          perturbation_function_holder=perturbation_function,
                                          use_cartesian_coordinates=False)

builder = SlabPerturbationBuilder()

### 2.2. Apply perturbation to the material

In [6]:
from mat3ra.made.tools.build.perturbation import create_perturbation

material_with_perturbation = create_perturbation(configuration, builder)

/Users/mat3ra/code/GREEN/api-examples/.venv/lib/python3.11/site-packages/mat3ra/made/tools/utils/perturbation.py:66: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  arc_length = quad(self._integrand, a, b, args=(coordinate, axis))[0]


### 2.3. Visualize the Material

In [7]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_perturbation, "title": f"Material with perturbation"},
           {"material": material_with_perturbation, "title": f"Material with perturbation", "rotation": "-90x"},
])

GridBox(children=(VBox(children=(Label(value='Ni900 - Original material - rotation: 0x,0y,0z', layout=Layout(a…

## 3. Create a Custom Perturbation
### 3.1. Set custom perturbation parameters
Provide a SymPy expression for the perturbation function. The expression should be a function of `x`, `y` and `z` variables.

In [8]:
import sympy as sp
from mat3ra.made.tools.build.perturbation import CellMatchingDistancePreservingSlabPerturbationBuilder
from mat3ra.made.tools.utils.perturbation import PerturbationFunctionHolder

x,y = sp.symbols('x y')
function = amplitude * sp.sin(2 * sp.pi * x / wavelength + phase) * sp.sin(2 * sp.pi * y / wavelength)

custom_perturbation_function = PerturbationFunctionHolder(function=function, variables=["x", "y"])
configuration_custom = PerturbationConfiguration(material=supercell,
                                                 perturbation_function_holder=custom_perturbation_function,
                                                 use_cartesian_coordinates=False)
distance_preserving_builder = CellMatchingDistancePreservingSlabPerturbationBuilder()

### 3.2. Apply perturbation to the material

In [9]:
material_with_custom_perturbation = create_perturbation(configuration_custom, distance_preserving_builder)

### 3.3. Visualize the Material

In [10]:
visualize([
    {"material": material_with_custom_perturbation, "title": f"Material with custom perturbation"},
    {"material": material_with_custom_perturbation, "title": f"Material with custom perturbation","rotation": "-90x"}
])

GridBox(children=(VBox(children=(Label(value='Ni900 - Material with custom perturbation - rotation: 0x,0y,0z',…

## 4. Pass data to the outside runtime

In [11]:
from utils.jupyterlite import set_materials

set_materials([material_with_perturbation, material_with_custom_perturbation])

Data for materials written to uploads/Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23 (Perturbation: SineWavePerturbationFunctionHolder).json
Data for materials written to uploads/Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23 (Perturbation: PerturbationFunctionHolder).json
